In [8]:
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torchvision import transforms, datasets

from classifiers.NerveNet_NU import NerveNET
from data_utils_NerveNet_NU import SegmentationData
from solver_NerveNet_NU import Solver
import transform_utils_NerveNet as tu
from dice_loss_NU import DiceLoss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The following calculates the **sizes of train, validation and test set**. Make sure it returns **True** by modifying the values in the dictionary 'nums'.

In [9]:
img_files = 'image_files.txt'
num_lines = sum(1 for line in open(img_files, 'r'))
nums = {'num_train' : int(0.6*num_lines)+2,
        'num_val'   : int(0.2*num_lines),
        'num_test' : int(0.2*num_lines)}

print(nums, num_lines)
print(np.sum(list(nums.values())) == num_lines)

{'num_train': 1278, 'num_val': 425, 'num_test': 425} 2128
True


**New in dataloader**: 
1. The dataloader has the new flag **binary_out**. If True, it will return a binary value that is if a nerve mask is contained in the target or not. Calling e.g. train_data[0] will return inputs and targets, where targets is now a dictionary with keys **'main'** and **'binary'**, if binary_out is True.
2. The text file that contains the image id's can now be in the git project directory.

In [10]:
input_dim = (1, 128, 128)

train_transforms = tu.Compose([tu.Resize(input_dim[1::]),
                               tu.RandomHorizontalFlip(),
                               tu.RandomVerticalFlip(),
                               tu.ToTensor()])

val_transforms = tu.Compose([tu.Resize(input_dim[1::]),
                               tu.ToTensor()])


train_data = SegmentationData(img_files, transform = train_transforms, mode = 'train', **nums, binary_out = True)
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=4)

val_data = SegmentationData(img_files,  transform = val_transforms, mode = 'val', **nums)
val_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=4)

**New in classifier**: 
1. NerveNet has a new **binary_out** flag, too. If True, the forward pass will calculate a binary output. Similarly the output of the forward pass is a dictionary with keys 'main' and 'binary'.
2. The argument **upsample_unit** is a switch between simple upsampling with bilinear interpolation and transposed convolutional layers. Must be either **'Upsample'** or **'ConvTranspose2d'**. Default is 'Upsample'.
3. **Xavier Normal Initialization** is automatically applied to all NerveNet layers.

In [11]:
model = NerveNET(input_dim, num_classes = 2, weight_scale = True, dropout = 0.15, 
                 binary_out = False, upsample_unit='Upsample')
#model = torch.load("models/NerveNet_NU.model")

**New in solver**:
1. Solver exspects new **binary_out** argument. Here, it should be a float value, that gives the weight of the loss of the binary output. The main output is implicitly weighted with one, so that **binary_out** should be **smaller than one**. If **binary_out** is equal to **zero**, **no binary loss** will be computed.
2. Train and validation are two seperate functions which seems to be more memory efficient and clearly structured.
3. Printing loss and accuracy were wrapped in seperate functions, too, to further enhance readability of the code.

**New in DiceLoss**:
1. DiceLoss now calculates and automatically detects the loss on **doubleclass input** (Batchsize, Classes = 2, H, W) with single class target (Batchsize, Classes = 1, H, W) by inverting the target for Class = 0 and adding losses for both classes.
2. The argument **classweights** receives weights for losses from class 0 and class 1, respectively. Default is [1, 1].
3. **Singleclass input** still doable. E.g. initialize model with **num_classes = 1** will work.



In [12]:
#for param in list(model.parameters()):
#    param.requires_grad = False
    
#for param in list(model.parameters())[0:50]:
#    param.requires_grad = True

solver = Solver(optim_args={"lr": 0.05, #0.0025, #1.e-3, #1.e-2
                            "betas": (0.9, 0.999),
                            "eps": 1e-8,
                            "weight_decay": 0.},
                loss_func = DiceLoss(classweights = [1, 1]), binary_out = 0)

model.train()
outputs = solver.train(model, train_loader, val_loader, log_nth=10, num_epochs=1)

START TRAIN.
|Iteration 10/320| TRAIN loss: 1.982


Process Process-14:
Process Process-13:
Process Process-16:
Traceback (most recent call last):
Process Process-15:
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3

KeyboardInterrupt: 

In [ ]:
model.save("models/NerveNet_NU.model")

In [ ]:
img_files = 'image_files.txt'


test_data = SegmentationData(img_files,  transform = val_transforms, mode = 'test', **nums)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=1)

In [ ]:
model.eval()

img, targets = test_data[5]

target = targets['main']
inputs = img.unsqueeze(0)
inputs = Variable(inputs)

model=model.cpu()

outputs = model.forward(inputs)
pred = outputs['main']
_, pred = torch.max(pred, 1)
pred = pred.squeeze().data.cpu().numpy()
img=np.squeeze(img)
target = target.squeeze().numpy()

fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[0].axis('off')
ax[1].imshow(target)
ax[1].axis('off')
ax[2].imshow(np.where(pred>0.1, 1, 0))
ax[2].axis('off')
plt.show()

In [ ]:
outputs['main']

In [ ]:
num_example_imgs = 10
plt.figure(figsize=(15, 5 * num_example_imgs))
for i, (img, targets) in enumerate(test_data[90:100]):
    
    target = targets['main']
    inputs = img.unsqueeze(0)
    inputs = Variable(inputs)

    outputs = model.forward(inputs)
    pred = outputs['main']
    _, pred = torch.max(pred, 1)
    pred = pred.squeeze().data.cpu().numpy()

    img=np.squeeze(img)
    target = target.squeeze().numpy()
        
    # img
    plt.subplot(num_example_imgs, 3, i * 3 + 1)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    if i == 0:
        plt.title("Input image")
    
    # target
    plt.subplot(num_example_imgs, 3, i * 3 + 2)
    plt.axis('off')
    plt.imshow(target, cmap='gray')
    if i == 0:
        plt.title("Target image")

    # pred
    plt.subplot(num_example_imgs, 3, i * 3 + 3)
    plt.axis('off')
    plt.imshow(pred, cmap='gray')
    if i == 0:
        plt.title("Prediction image")
    
plt.show()

In [ ]:
def dice_coefficient(ground_truth, predicted):
    gt = ground_truth
    p = predicted
    if np.sum(p) + np.sum(gt) == 0:
        return 1
    else:
        dice = np.sum(p[gt==1])*2.0 / (np.sum(p) + np.sum(gt))
        return dice

test_scores = []
model.eval()
for inputs, targets in test_loader:
    inputs, targets = Variable(inputs), Variable(targets['main'])
    if model.is_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    
    outputs = model.forward(inputs)
    pred = outputs['main']
    _, pred = torch.max(pred, 1)
    pred = pred.squeeze().data.cpu().numpy()
    test_scores.append(dice_coefficient(np.squeeze(targets.data.numpy()), pred))
    
model.train()
np.mean(test_scores)

In [ ]:
x = np.array(test_scores)
x[x==1] = 0
np.argmax(x)